In [1]:
#-*- coding=utf-8 -*-
import numpy as np
import numpy as np
import pandas as pd
import jieba
from tqdm import tqdm
import re
import gensim

word_size = 128 #词向量维度
maxlen = 80 #句子截断长度

In [2]:
dfTrain = pd.read_table('Train.csv',sep='\t',index_col=False)

not_cuts = re.compile(u'([\da-zA-Z \.]+)|《(.*?)》|“(.{1,10})”')
re_replace = re.compile(u'[^\u4e00-\u9fa50-9a-zA-Z《》\(\)（）“”·\.]')
def mycut(s):
    result = []
    j = 0
    #s = re_replace.sub(' ', s)
    s=re.sub('---| ','',s)
    for i in not_cuts.finditer(s):
        result.extend(jieba.lcut(s[j:i.start()], HMM=False))
        if s[i.start()] in [u'《', u'“']:
            result.extend([s[i.start()], s[i.start()+1:i.end()-1], s[i.end()-1]])
        else:
            result.append(s[i.start():i.end()])
        j = i.end()
    result.extend(jieba.lcut(s[j:], HMM=False))
    return result

In [3]:
dfTrain['words'] = dfTrain['Content'].apply(mycut)
#l=dfTrain['words'][18]
#for i in range(len(l)):
#    print l[i]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/5w/0865ltkd337gg3z4gyvndhym0000gn/T/jieba.cache
Loading model cost 0.473 seconds.
Prefix dict has been built succesfully.


In [4]:
dfTest = pd.read_table('Test.csv',sep='\t',index_col=False)
dfTest['words'] = dfTest['Content'].apply(mycut)

In [5]:
#读取Label.csv
dfLabel = pd.read_table('Label.csv',sep='\t',index_col=False)
gb = dfLabel.groupby("SentenceId",as_index=False)
dfLabelGb = gb.aggregate(lambda x: list(tuple(x)))
dfTrainLabel = pd.merge(dfTrain,dfLabelGb)

In [6]:
import sys   
reload(sys) # Python2.5 初始化后会删除 sys.setdefaultencoding 这个方法，我们需要重新载入   
sys.setdefaultencoding('utf-8') 

7428it [00:06, 1085.96it/s]

In [7]:
#利用lable结果（比如“大众“），来标记原句中此关键词出现的位置
def label(k): #将输出结果转换为标签序列
    s = dfTrainLabel['words'][k]
    r = ['0']*len(s)
    for i in range(len(s)):
        for j in dfTrainLabel['View'][k]:
            if s[i] in j:
                r[i] = '1'
                break
    s = ''.join(r)
    r = [0]*len(s)
    for i in re.finditer('1+', s):
        if i.end() - i.start() > 1:
            r[i.start()] = 2
            r[i.end()-1] = 4
            for j in range(i.start()+1, i.end()-1):
                r[j] = 3
        else:
            r[i.start()] = 1
    return r

dfTrainLabel['label'] = map(label, tqdm(iter(dfTrainLabel.index))) #输出tags

In [8]:
idx = range(len(dfTrainLabel))
dfTrainLabel.index = idx
np.random.shuffle(idx)
dfTrainLabel = dfTrainLabel.loc[idx]
dfTrainLabel.index = range(len(dfTrainLabel))

In [9]:
word2vec = gensim.models.Word2Vec(dfTrain['words'].append(dfTest['words']), 
                                  min_count=1, 
                                  size=word_size, 
                                  workers=20,
                                  iter=20,
                                  window=8,
                                  negative=8,
                                  sg=1)

In [10]:
word2vec.save('word2vec_words_final.model')
word2vec.init_sims(replace=True)

In [11]:
from keras.layers import Dense, LSTM, Lambda, TimeDistributed, Input, Masking, Bidirectional
from keras.models import Model
from keras.utils import np_utils
from keras.regularizers import activity_l1 #通过L1正则项，使得输出更加稀疏
 
sequence = Input(shape=(maxlen, word_size))
mask = Masking(mask_value=0.)(sequence)
blstm = Bidirectional(LSTM(64, return_sequences=True), merge_mode='sum')(mask)
blstm = Bidirectional(LSTM(32, return_sequences=True), merge_mode='sum')(blstm)
output = TimeDistributed(Dense(5, activation='softmax'))(blstm)
model = Model(input=sequence, output=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 

In [12]:
'''
gen_matrix实现从分词后的list来输出训练样本
gen_target实现将输出序列转换为one hot形式的目标
超过maxlen则截断，不足补0
'''
gen_matrix = lambda z: np.vstack((word2vec[z[:maxlen]], np.zeros((maxlen-len(z[:maxlen]), word_size))))
gen_target = lambda z: np_utils.to_categorical(np.array(z[:maxlen] + [0]*(maxlen-len(z[:maxlen]))), 5)
 

In [13]:
#从节省内存的角度，通过生成器的方式来训练
def data_generator(data, targets, batch_size): 
    idx = np.arange(len(data))
    np.random.shuffle(idx)
    batches = [idx[range(batch_size*i, min(len(data), batch_size*(i+1)))] for i in range(len(data)/batch_size+1)]
    while True:
        for i in batches:
            xx, yy = np.array(map(gen_matrix, data[i])), np.array(map(gen_target, targets[i]))
            yield (xx, yy)
 
batch_size = 1024
history = model.fit_generator(data_generator(dfTrainLabel['words'], dfTrainLabel['label'], batch_size), 
                              samples_per_epoch=len(dfTrainLabel), nb_epoch=10)
model.save_weights('words_seq2seq_final_1.model')

In [14]:
def predict_data(data, batch_size):
    batches = [range(batch_size*i, min(len(data), batch_size*(i+1))) for i in range(len(data)/batch_size+1)]
    p = model.predict(np.array(map(gen_matrix, data[batches[0]])), verbose=1)
    for i in batches[1:]:
        print min(i), 'done.'
        p = np.vstack((p, model.predict(np.array(map(gen_matrix, data[i])), verbose=1)))
    return p
 
dfTrainLabel['predict'] = list(predict_data(dfTrainLabel['words'], batch_size))
dfTest['predict'] = list(predict_data(dfTest['words'], batch_size))

In [15]:
'''
动态规划部分：
1、zy是转移矩阵，用了对数概率；概率的数值是大概估计的，事实上，这个数值的精确意义不是很大。
2、viterbi是动态规划算法。
'''
zy = {'00':0.15, 
      '01':0.15, 
      '02':0.7, 
      '10':1.0, 
      '23':0.5, 
      '24':0.5,
      '33':0.5,
      '34':0.5, 
      '40':1.0
     }
 
zy = {i:np.log(zy[i]) for i in zy.keys()}

In [16]:
def viterbi(nodes):
    paths = nodes[0]
    for l in range(1,len(nodes)):
        paths_ = paths.copy()
        paths = {}
        for i in nodes[l].keys():
            nows = {}
            for j in paths_.keys():
                if j[-1]+i in zy.keys():
                    nows[j+i]= paths_[j]+nodes[l][i]+zy[j[-1]+i]
            k = np.argmax(nows.values())
            paths[nows.keys()[k]] = nows.values()[k]
    return paths.keys()[np.argmax(paths.values())]

In [17]:
'''
整理输出结果，即生成提交数据所需要的格式。
整个过程包括：动态规划、结果提取。
'''
 
def predict(i):
    nodes = [dict(zip(['0','1','2','3','4'], k)) for k in np.log(dfTest['predict'][i][:len(dfTest['words'][i])])]
    r = viterbi(nodes)
    result = []
    words = dfTest['words'][i]
    for j in re.finditer('2.*?4|1', r):
        result.append((''.join(words[j.start():j.end()]), np.mean([nodes[k][r[k]] for k in range(j.start(),j.end())])))
    if result:
        result = pd.DataFrame(result)
        return [result[0][result[1].argmax()]]
    else:
        return result
 
dfTest['View'] = map(predict, tqdm(iter(dfTest.index), desc=u'第一次预测'))
 

In [18]:
'''
导出提交的JSON格式
'''
gen = lambda i:'[{"Content": "'+dfTest.iloc[i]['Content']+'", "View": ["'+''.join(dfTest.iloc[i]['View'])+'"]}]'
ssss = map(gen, tqdm(range(len(dfTest))))
result='\n'.join(ssss)
import codecs
f=codecs.open('result1.txt', 'w', encoding='utf-8')
f.write(result)
f.close()
import os
os.system('rm result1.zip')
os.system('zip result1.zip result1.txt')

0

In [19]:
dfTest

,SentenceId,Content,words,predict,View
0,17063,我大汽车圈如今面临互联网造车的挤兑，能体现咱们牛叉制造业优势的不是别的，就是深湛难以短期复制...,"[我, 大, 汽车, 圈, 如今, 面临, 互联网, 造, 车, 的, 挤兑, ，, 能, ...","[[0.722333, 0.0635106, 0.0943637, 0.0496921, 0...",[]
1,10843,其中自主品牌仍然增速相对更高：东风乘用车和东风小康涨幅分列前两名，分别高达76.69%和70...,"[其中, 自主, 品牌, 仍然, 增速, 相对, 更, 高, ：, 东风, 乘用车, 和, ...","[[0.705551, 0.0698167, 0.0995727, 0.051416, 0....",[]
2,6903,18英寸凌渡GTS专属双色铝合金轮圈造型硬朗，红色刹车卡钳，彰显了个性。,"[18, 英寸, 凌, 渡, GTS, 专属, 双色, 铝合金, 轮圈, 造型, 硬朗, ，...","[[0.645197, 0.0731939, 0.125289, 0.0645321, 0....",[英寸凌]
3,13467,创驰蓝天六速自动变速器7英寸中央信息显示屏/悦联系统宝马式控制旋钮非常高大上杯槽深度一般，放...,"[创, 驰, 蓝天, 六, 速, 自动, 变速器, 7, 英寸, 中央, 信息, 显示屏, ...","[[0.659107, 0.0713703, 0.120181, 0.0580479, 0....",[]
4,13991,这篇文章汇聚了内布拉斯加——林肯的教授和其他教师们的教学经验，提出课程的顺利进行，靠的不仅是...,"[这, 篇文章, 汇聚, 了, 内布拉斯加, —, —, 林肯, 的, 教授, 和, 其他,...","[[0.719243, 0.0655289, 0.0940337, 0.0534457, 0...",[]
5,21442,一开始是看上了途观、汉兰达、昂科威和国光这四个车的，这段时间自己又比较了下，排除了途观和汉兰达。,"[一, 开始, 是, 看, 上, 了, 途, 观, 、, 汉, 兰, 达, 、, 昂, 科,...","[[0.734321, 0.0646501, 0.0881634, 0.0452277, 0...",[]
6,21402,穆勒称波斯米亚零部件制造商Prevent Group旗下两家子公司因合约争议导致其供货中断，...,"[穆勒, 称, 波斯, 米亚, 零部件, 制造商, PreventGroup, 旗下, 两家...","[[0.696425, 0.0708035, 0.105669, 0.052251, 0.0...",[]
7,19857,并且长安悦翔V7的车头远观挺顺眼，而细细品味又有几处是我对于有设计感的车型感冒的地方。,"[并且, 长安, 悦, 翔, V7, 的, 车头, 远, 观, 挺, 顺眼, ，, 而, 细...","[[0.706274, 0.0653643, 0.0997958, 0.050933, 0....",[]
8,16427,而同样经过熏黑处理的灯组，可以被视为威朗GS车型上独具匠心的一部分。,"[而, 同样, 经过, 熏黑, 处理, 的, 灯, 组, ，, 可以, 被, 视为, 威, ...","[[0.698213, 0.0649816, 0.104786, 0.0535063, 0....",[]
9,27350,Yeti的后排座椅提供了两套ISOFIX儿童安全座椅固定装置，儿童安全座椅安装方便，固定紧密...,"[Yeti, 的, 后排, 座椅, 提供, 了, 两套, ISOFIX, 儿童, 安全, 座...","[[0.69332, 0.0653962, 0.102544, 0.0568995, 0.0...",[]
